In [2]:
%pip install Bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import random
import time
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import PairwiseAligner
from Bio.Blast import NCBIWWW, NCBIXML

def random_deletion(seq, num_deletions):
    seq = list(seq)
    for _ in range(num_deletions):
        del seq[random.randint(0, len(seq) - 1)]
    return ''.join(seq)

def random_insertion(seq, num_insertions):
    seq = list(seq)
    for _ in range(num_insertions):
        seq.insert(random.randint(0, len(seq) - 1), random.choice("ACGT"))
    return ''.join(seq)

def random_mutation(seq, num_mutations):
    seq = list(seq)
    for _ in range(num_mutations):
        seq[random.randint(0, len(seq) - 1)] = random.choice("ACGT")
    return ''.join(seq)

def repeat_subsequence(seq, start, length):
    subseq = seq[start:start + length]
    return seq[:start + length] + subseq + seq[start + length:]

def delete_subsequence(seq, start, length):
    return seq[:start] + seq[start + length:]

# FRA10B_datasets

original_seq = """ATATATATATATTTTTTTATATACATTATATATTTATATATAAATAAAATTTTTATTACATATATACATT
ATATACTTATACATAATTTTTTATTTTATTATATATTTATATATCTCTATTATATACATTATATAATTAT
ATATCATATATTTATATATGATATAATTATATATCATATATTTATATATGATATAATTATATATCATATA
TTTATATATGATATAATTATATATCATATATTTATATATGATATAATATATATGTATATAATATATAAAT
ATATATAAATATATAATATATATTTATATATATTTTATATATATACACATATTATATATGTGTATATTAT
ATAATATATACACATATTATATATGTGTATATTATATAATATACACATATATCATATAATTATATATTAT
ATAATATATATATTATATATGTATATATTATATATTATATAATGTATATATTATATATTATATAATGTAT
ATATTATATCATATATAATATATTATATATATGTATATATTATATATGATATATAATATATAATATATAT
GTATATATTATATATGATATAATATATACATATTATATATGTATATATTATATATGATATAATATATACA
TATTATATATGTATATATTATATATGATATAATATATACATATTATATGTATATATTATATAATATATAC
ATATATATGTATATATTATATATGTATATATTTATATACATATATAATATATGTATATATTATGTATATA
TTTATATACATATATTATATATAATATATATAATATATACGTATATAATATATAATACACTATATAATAT
ATACGTATATAATATATATATTAGATATAATATATACGTATATAATATATAATACACTATATAATATATA
CGTATATAATACATATATACTATATAATATATACGTATATAATACATATATACCATATAATATATACGTA
TATAATACATGTATTATATACGTATATAATATATATGTATACAATATATACGTATATAATATATATGTAT
ATATTATATAAATTATATATTATATATATGTATTATATAAATTATATATTTTTT"""
X = random_deletion(original_seq, len(original_seq) - 400)

def generate_variations(X, k):
    X1 = random_insertion(X, k)
    X1 = random_deletion(X1, k)
    X1 = random_mutation(X1, k)
    X1 = repeat_subsequence(X1, 2 * k, k)
    X1 = delete_subsequence(X1, 400 - 2 * k, k)

    X2 = random_insertion(X, k)
    X2 = random_deletion(X2, k)
    X2 = random_mutation(X2, k)
    X2 = repeat_subsequence(X2, 400 - 2 * k, k)
    X2 = delete_subsequence(X2, 2 * k, k)

    return X1, X2

for k in [10, 20, 30, 40, 50]:
    X1, X2 = generate_variations(X, k)

    start_time_nw = time.time()
    aligner = PairwiseAligner()
    aligner.mode = 'global'
    alignment = aligner.align(X1, X2)
    nw_score = alignment.score
    duration_nw = time.time() - start_time_nw

    start_time_blast = time.time()
    result_handle = NCBIWWW.qblast("blastn", "nt", X1)
    blast_record = NCBIXML.read(result_handle)
    blast_score = max(hsp.score for alignment in blast_record.alignments for hsp in alignment.hsps)
    duration_blast = time.time() - start_time_blast

    print(f"k = {k}")
    print(f"Needleman-Wunsch score: {nw_score}, Time taken: {duration_nw} seconds")
    print(f"BLAST score: {blast_score}, Time taken: {duration_blast} seconds")
    print("-" * 40)


k = 10
Needleman-Wunsch score: 348.0, Time taken: 0.0021347999572753906 seconds
BLAST score: 136.0, Time taken: 66.92744135856628 seconds
----------------------------------------
k = 20
Needleman-Wunsch score: 319.0, Time taken: 0.0019021034240722656 seconds
BLAST score: 149.0, Time taken: 76.45354580879211 seconds
----------------------------------------
k = 30
Needleman-Wunsch score: 298.0, Time taken: 0.0021550655364990234 seconds
BLAST score: 87.0, Time taken: 138.22266912460327 seconds
----------------------------------------
k = 40
Needleman-Wunsch score: 294.0, Time taken: 0.001962900161743164 seconds
BLAST score: 88.0, Time taken: 141.95643520355225 seconds
----------------------------------------
k = 50
Needleman-Wunsch score: 290.0, Time taken: 0.0033791065216064453 seconds
BLAST score: 75.0, Time taken: 79.69526410102844 seconds
----------------------------------------


In [5]:
import random
import time
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import PairwiseAligner
from Bio.Blast import NCBIWWW, NCBIXML

def random_deletion(seq, num_deletions):
    seq = list(seq)
    for _ in range(num_deletions):
        del seq[random.randint(0, len(seq) - 1)]
    return ''.join(seq)

def random_insertion(seq, num_insertions):
    seq = list(seq)
    for _ in range(num_insertions):
        seq.insert(random.randint(0, len(seq) - 1), random.choice("ACGT"))
    return ''.join(seq)

def random_mutation(seq, num_mutations):
    seq = list(seq)
    for _ in range(num_mutations):
        seq[random.randint(0, len(seq) - 1)] = random.choice("ACGT")
    return ''.join(seq)

def repeat_subsequence(seq, start, length):
    subseq = seq[start:start + length]
    return seq[:start + length] + subseq + seq[start + length:]

def delete_subsequence(seq, start, length):
    return seq[:start] + seq[start + length:]

#FGF4_datasets

original_seq = """ATGGCGGGGCCCGGGGCGGCCGCGGCGGCGCTGCTCCCGGCGGTGCTGCTGGCCGTGCTGGCGCCCTGGG
CCGGCCGAGGGGGCGCCGCCGCGCCCACCGCCCCCAACGGCACGCTGGAGGCCGAGCTGGAGCGCCGCTG
GGAGAGCCTGGTGGCGCGCTCGCTGGCGCGCCTGCCGGTGGCCGCGCAGCCCAAGGAGGCTGCCGTCCAG
AGCGGCGCCGGCGACTACTTGCTGGGCATCAAGCGGTTGCGGAGGCTGTACTGCAACGTGGGCATCGGCT
TCCACCTCCAGGTGCTCCCCGACGGCCGCATCGGCGGCGTGCACGCGGACACGAGTGACAGTGAGTGGGG
CGGGCCGGACGGGGAGGGTCGGGGCCCGGGACAGCTGCCGCCCCTTCCAGCTCGCACAGGAAGGTCTCGC
CGGCACCTGCTCTTGGGCTATGGGAACCGGAGCGGACTCGGGGTCCGGTCCCCTCTGGCGTAAAGACGCC
GGGGACGCCAAACACTCTTGGCTTTGCGCAACCCGGCGGCTGAGCCCGCCTCCCGCCTTCAGAAGTCCGG
AATCTGGGTGCCGGATTCCAGCGCCTTCGCTCGTGGGCACCGGGAGCCCGGCGCAGCCACCGGCAACCCG
CAGTGGGCGGCTCGCGCGCCCACGTCGGGGTCGGACTAGGGGGCCGCAGGGGGCCACCCCGCTCCTGCAG
TCCCGGACTGGGCGCGGCCGGCGCGGGGCCGGGACAGGGGCCGAGCCCGCGGGGCTCCTGGGGCCCATGG
GGGGCGGCCTGCACCGCGGAGGGAGCAGGGCCATTTAGATTCATCTCCACCCCCAGAAGTGGGGGTCCCG
GGGGGGAGGAGGGGCCCAGGTGGGAAGCTCGGGCCGAGGGTCCGGTGCCCGCGAATGACCCGCGCCCCGC
CCCCAGGCCTGCTGGAGCTGTCGCCGGTGGAGCGCGGGGTGGTGAGCATTTTCGGGGTGGCCAGCCGGTT
CTTCGTGGCCATGAGCAGCCGGGGCAGGCTGTACGGCTCGGTGAGTACCGTGGGCCCCGCCGAGCCGGGC
AGCATCGTGACGCTTCCAGGCCGTCCGACGCCAGGATCGTGGAGCTGCGGTGGGGGCCGCGGGATTGAAA
GTAACCCCGCTCCTGAGGCAGTGCCGCTCCCCCTTGTCGTTCAGTCACTCAGTCCTGTTGGATTTCTTTG
GCGATCCTGTAGACCCTCCGCTGCCAGGCTCCCCTGTCCATGGGATTTCCCAGGCCAGAATCCTGGAGTG
AGTGAGTGGGTGGTCAGTTTCCTTCTCCAGAGGATCTTCCTGAACCCGGGGATCCAACCTGCATCTCCTG
CATTGGCAGGAGGATTCTTTACCACCGAGCCACCAGGGACGCCCACCAGCCCAAAGGAGAGGCCCAGCCC
CAAGACAGGGACAGCGCTCCCGCCATCCTTGAGCGGTCCCTTGCCAGTGGGTCAAGGATGCCATCGGGAG
CCGCTGCCTGGAGCCCCTGGGGCGGGTCCCAGTGGGCAGGACGGCAGTCGTTAGGTGGACCCAGTGAAGA
CGCACTGGTCTGGCTTCCAGGTGCCGGGCAGGTGCCTGGAGTCGCGGTGGCTGGGGACTGGCGGGGTCCC
TCTCCTCACCCCGTGAAAGTCTGGGTACCCTGGCTTCTCAGGAGGGACCTCCCAGCCAAGGAGGGGCCGG
GAGAGAGAGGAGTCCCTGACCACTGCCCCAGAGGTCCCTTGGCCCGCGATGGCCCCTGACCCGGCTCCTC
CTTCCCTCCCAGCCTTTCTTCACCGACGAGTGCAGGTTCAGAGAGATCCTCCTCCCCAACAACTACAACG
CCTACGAGTGCGACCGGCACCCCGGCATGTTCATCGCCCTGAGCAAGAACGGCAAGGCCAAGAAGGGGAA
CCGGGTGTCCCCCACCATGAAGGTCACCCACTTCCTCCCCAGGCTGTGA"""
X = random_deletion(original_seq, len(original_seq) - 400)

def generate_variations(X, k):
    X1 = random_insertion(X, k)
    X1 = random_deletion(X1, k)
    X1 = random_mutation(X1, k)
    X1 = repeat_subsequence(X1, 2 * k, k)
    X1 = delete_subsequence(X1, 400 - 2 * k, k)

    X2 = random_insertion(X, k)
    X2 = random_deletion(X2, k)
    X2 = random_mutation(X2, k)
    X2 = repeat_subsequence(X2, 400 - 2 * k, k)
    X2 = delete_subsequence(X2, 2 * k, k)

    return X1, X2

for k in [10, 20, 30, 40, 50]:
    X1, X2 = generate_variations(X, k)

    start_time_nw = time.time()
    aligner = PairwiseAligner()
    aligner.mode = 'global'
    alignment = aligner.align(X1, X2)
    nw_score = alignment.score
    duration_nw = time.time() - start_time_nw

    start_time_blast = time.time()
    result_handle = NCBIWWW.qblast("blastn", "nt", X1)
    blast_record = NCBIXML.read(result_handle)
    blast_score = max(hsp.score for alignment in blast_record.alignments for hsp in alignment.hsps)
    duration_blast = time.time() - start_time_blast

    print(f"k = {k}")
    print(f"Needleman-Wunsch score: {nw_score}, Time taken: {duration_nw} seconds")
    print(f"BLAST score: {blast_score}, Time taken: {duration_blast} seconds")
    print("-" * 40)


k = 10
Needleman-Wunsch score: 349.0, Time taken: 0.0019969940185546875 seconds
BLAST score: 58.0, Time taken: 121.76336312294006 seconds
----------------------------------------
k = 20
Needleman-Wunsch score: 313.0, Time taken: 0.0019681453704833984 seconds
BLAST score: 60.0, Time taken: 139.73294162750244 seconds
----------------------------------------
k = 30
Needleman-Wunsch score: 295.0, Time taken: 0.003635883331298828 seconds
BLAST score: 54.0, Time taken: 320.1296570301056 seconds
----------------------------------------
k = 40
Needleman-Wunsch score: 277.0, Time taken: 0.003556966781616211 seconds
BLAST score: 52.0, Time taken: 79.88582015037537 seconds
----------------------------------------
k = 50
Needleman-Wunsch score: 276.0, Time taken: 0.0023958683013916016 seconds
BLAST score: 53.0, Time taken: 140.67147278785706 seconds
----------------------------------------


In [6]:
import random
import time
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import PairwiseAligner
from Bio.Blast import NCBIWWW, NCBIXML

def random_deletion(seq, num_deletions):
    seq = list(seq)
    for _ in range(num_deletions):
        del seq[random.randint(0, len(seq) - 1)]
    return ''.join(seq)

def random_insertion(seq, num_insertions):
    seq = list(seq)
    for _ in range(num_insertions):
        seq.insert(random.randint(0, len(seq) - 1), random.choice("ACGT"))
    return ''.join(seq)

def random_mutation(seq, num_mutations):
    seq = list(seq)
    for _ in range(num_mutations):
        seq[random.randint(0, len(seq) - 1)] = random.choice("ACGT")
    return ''.join(seq)

def repeat_subsequence(seq, start, length):
    subseq = seq[start:start + length]
    return seq[:start + length] + subseq + seq[start + length:]

def delete_subsequence(seq, start, length):
    return seq[:start] + seq[start + length:]

#Cnp_datasets

original_seq = """AGGCTATGAAAGGGACCCTGCGTGGGGGGGCCCCGGACACCCGGAGACGCCCGTGTCCCTCACGCAGGCG
GGCGGCCCCGGAGACATAGTACCCGCAAAGGCGGTGACGGCGGTGCGCCCACTCATCATGGTGAGCGCAG
CGCGTGCGTGGGGGCCGGGGCGGGAGAGAGGCTAACTGAAAGCGAGATTCTTGGGGATGCATGGGCAGAA
GTAGTTTGCAAATGAGGACACCAGGTTCCGGAGTTGAAGTTGCCTCTGGGTCCTCCAAGGCAGCCCTAAG
GCAAGGGGCGCGAGCGGGGATATTTTGGGGGAGTCTTAGTGTCTTTGGAGAAGGAAATATCGCCGCAGGG
AAGTGAAAGTCGATGCTCCTTGGCGACACCAGCTGGCAAATGGGTCTGGGTAGCACCAGAGAGTGCAGGG
CTGTTGTGGAGCCCGCGCTGGGAGTGGGTTCCTGCACAAGGCTATAGAGAAGCCAGAAGCCAGCTAAATT
CTCAGGGGCGGAAGCCGTTGTAGATGACAACTGAAACCGCTATGTCTTGTTTTTGTCAGAGAGGTGCTGC
TAACACACAGCAGAATAGGGGGCACAGATCTCTGAATACAGGTGCCCTGTGCCCTAATCCTACAGATTGG
GGGAGGGGGCGCACTGAGAGAGCAGAGGAGGAGGTGGTTGTGAATATGCCCTTTGTCCAGTGCCCTCCTG
CCCTGCCCCCCACCAGCCCTCTGTGGGACCATTGTCCCCCTATACCCAGACAAGAGAGTATTATCTGTTG
GTGCCATTGTTGGGTGGAGGGAGGCGGTCTTTGGGTCACCCTCCCCTGCCCTCACCCTCCTCTGCACACT
CAAGAGAGCCAAGGCAGCTGTGGCTTTGCCCATACATAGAGCCCCAGGCCTCCAAACAGGACATCCTTAA
GAGGTGCCAAGAATATATAAGTTAGGGCTGACAGGGAGCCACAGGCACTTGCTGCCCTTGCGTGCCCTGT
CACCAAGGTGTTCTGAGGGGGAGCAGAGAGTGATCCTTGGAGCCAGAGACTAAGAGGTTAGGTAGCTTCA
GAAAGATTCCCTCTACCTTACAACCCAGCTGTTGAAGATACTGAAGGATCTACTGCGAGGAAGAAAATCC
ACCATTCCCCTCCGGGGAAAGCATCCACCATTCAAGGGTGCTTTAAGTTAGGCTTGGGGATTCCTCAACT
GACAAGCGGTCCCCCTCTGGGAGAACTCGGCCAGAGACTAGGGTGTGGTGCTTGCGGGCTGGCTTTGAGG
AGCCGGGGCTGACATCTCCTCCTCCCTTCTTACACAGAACACAAGCTTTACCCGCAAAAGCCACACATTC
CTGCCCAAGCTCTTCTTCAGGAAAATGTCATCCTCAGGAGCAAAGGAGAAGCCAGAGCTGCAGTTCCCTT
TCCTTCAAGATGAGGACACGGTGGCTACCCTCCACGAGTGCAAGACGCTATTCATCCTGCGCGGCCTGCC
GGGCAGCGGCAAGTCCACACTGGCTCGGCTCATCCTGGAGAAGTACCACGACGGCACCAAGATGGTGTCC
GCTGATGCTTACAAGATCATCCCTGGCTCTCGGGCAGACTTCTCCGAGGCGTACAAGCGTCTAGACGAGG
ACCTGGCTGGCTACTGCCGCCGGGACATCAGGGTTCTTGTGCTTGATGATACCAACCACGAGCGGGAGCG
GCTGGATCAGCTTTTTGAAATGGCAGACCAGTATCAGTACCAGGTGGTGCTGGTGGAGCCCAAGACAGCG
TGGCGACTAGACTGTGCCCAGCTCAAGGAGAAGAACCAATGGCAGCTGTCGGCCGATGACCTGAAGAAGC
TGAAGCCCGGGCTGGAGAAGGACTTTCTGCCACTCTACTTTGGCTGGTTCCTGACCAAAAAGAGTTCTGA
GACCCTCCGAAAAGCTGGCCAGGTCTTTCTGGAGGAGCTGGGGAATCACAAGGCCTTCAAGAAAGAGCTT
CGACACTGTAGGTGGATGTGGGGTGGGAGCAGGGCAGCTAAAGCCTTGTACTGTTTGCCTGTGATGGAGA
GCCAGGCAGCGCAGCTGCTGAGCTCAAAGGATGGGGTTGACGGGTGGAGGGTACAGGTAGCAGCTTCTAG
GACTGGCCAGAAGCAAGGCGTCAGTGGAGAAGGGCATGCCCATAGCTCCTTTCTTGCCTTCCTCTTCCCC
CCCCAGCACCAGGATATCCTCCCCGTTTGGCTCTAGACCTCCATCTGAGTCTTGGAGGTGGGAAAAAACT
GTAATTCTCTTCGCCATATGCTATAAGCTCCTCGGTGGAGGGCGGAGGGAGACCAAGGCTGTCAAGATCA
GTTTTAGCAGCAGAATATATGTTTCTAAGCAGAAGGTGAGGTGGAAACCTAAACTGCAAAAAGTAGCAAG
GCCAAGGCTGTGTGACTTTCCAGAGAATGGGTCTTGGGCTCCCCCATGGCATCTCAAGCTGTAGAGAACC
CAGCTTCATGCCTTCCAGAATATTCTCTCCATATAAGAGACATGCCCAGAAAGGATATAAGAGTTGCCAC
CTGTACCTGGGTAGGAGCCAGGAGGATTCTTTCCTGGTAAGGGCTGTTGGCTCCAATGAGTGGATAGTCT
GGGGTAGGGTAGAGTCCTCAGTGGTCCCATTCCCCCAGTGTCCCACAGGTAGTTGAGTCTTGGGTGTTTT
GTTTTGTTTTGGAGATAAAAGTCTATGTAGCCTTCCATGGCGGGCCTGGAACTCACTTCATCCATCAGGC
TGGTCTCTGACTTGCAGTAATTCTCCTGCTTTTACCTTCCCCAGTACTGGAGACTTAGGCACTCTCAGTT
TTAGCTGGCCTGTAAACAAACTGCTTCATTCTGTTGTTCGACTGCTCAGAGGCAAACAGGTCCTTTGTGC
CCATGTGAGTTTTCAGAAAACTTATTTCTAGGCTCCTAAGACTCAAACTTTTACAAGGAAGATGTTAGTT
GAGAGCCAGGCCACTTTTTGTATAGAGGGGCAACGATCAGGGACAGGGCTTCTGGCCAGAGCTGTTGGTG
CTGGCTCGGGCATGATTACTGTTTGATGCTGGCTTTGGCCACAAACACCAGAATGCTTTTCTGCTCAGAG
TGTGACAGGGGACATCAGAAACAATAACAGGACGTGCTCAGAATAGAGGACCCCTTGAAGCATTAGCATT
TGGGAAGGAATGGAAAAGCTAGAGACCAAATTCAGCCACCAAGCTTGGCCTCTGTTTGGAAGCCCAGCTT
TTGACATCCTCCTGGTCCTCAGCCACCTCGTCCCACTGTTCCTAGGACCCTTCCTGTTCCAGGCCCTTGG
CTTGCTCTCCTCCTCCACGCATGCCTTGCAGGGCATGAGGGGTCAGTCACCACTGGCTGAAATGGTTGCT
GTAAATCTCAGAGGCCCTGAGGAATTTGGGGATATGGCAACTGGAACTGTCACAGTGGACTGACAAGTCA
CCCACCCACGTGTCCCAGGTGGCTAGTAAGTGGCAGGATGGGTTTCATTTCCCCATGTACCACCTGGTTC
CCCTTGCCTTTGGCTTGTATGGCCTAGCCCTGTGTTACATATGCCTGAGAACTCAAGTAGAGATATAAAG
CTCTATACGCTGTAACGCCAGTATGGTTTGGTTTGTTCCCTGTCCAGCATTAAAGCCAGGACCCTATGCA
TGCTGGGCAAGGATCCCAGCTGCCCTCCCCCGACCCCGTTGAGAACTCCAGATCTTCCTGCATCTATTTT
CCTGGAGCTAGGGTTACATGTGTATGTCACCATGCTTGCTTTACGCAGTGCTAGTGCCGGGGACTCAACC
TGGGGTGTTGGCGCAGGCCAGGCCAGGCCAGCACTCTGCCCACTGACCTTCGTCTCCAGCTAGCAAATAA
GCGAAGCACAGCAGAGCCACCCGAGGACTTCTTAGTAATGGCCCTGGTTTTGGTTTGCAGTCTCCGAGAT
AGGTGTTGGTAGGCCCAGCTCCCCACCTTTGATGGACCCTTTTCTCTTCTGCAGTTATTTCTGGAGATGA
ACCCAAGGAGAAGCTTGAGCTGGTCAGCTACTTTGGAAAGAGACCTCCAGGTGTGCTGCACTGTACAACC
AAATTCTGTGACTACGGGAAGGCCGCTGGGGCAGAAGAATACGCCCAGCAGGAGGTGAGTCCTCCTCCCC
AGGGACGCCCAGGTGGAGGTGGGGAGGGTAAGTGGGTTGGAGAAGTGTGGCCTCCAGTCTTTGGTGTAGT
CTGTAATCTTGTTATCCTCTCCTAGGGCCTCACTAGTCCCATCTCTCCCCAGGGCCTGCCAAAGGGCCTT
CTCCTTGACATAGATTTATGGATTTATGCCAGGCAGCAGCAGGTTCTTGCTCTGGCTTCTCAAACCCCTG
TTAAGTTTCAGAGTCTCAAATGCTGAGAGTCCTGTATCCCAGGGAGCCTGTTCTACTGGCCTTGCTCCAG
AGGGAGTGGCTCAGGATACCAAGCCACCCATTCCTGCACCATAGCACACAGCCCATGGCCAGATACAGGC
ATGCTGTCTACTTCCTAACTTCCTTTACCATGGCCTTCCCCGCAGCCATTCACCTGCTTGCTTACTGGCC
AACCTCAGGGGAAGCGGCAGCTAAGGGGCCTTGTCTGTTCGTGTTCTCAGTGTAATGGCTGAATATGTAC
CGTCGCTCCTTCCTACTGTCCTACTTCCTACTGTCCGCTCACTAAGCACCTGCCACATGCTGCTGCTGGA
ACAGAAGTTCATGAACAAAAAGATTTTACAGTTAGGTCACATGAACTCACGGAAGAGGCTCAGGGTAGGC
AGAGGTAACCAGATGTAGACCCCAGCGTGTTGGTGGAGGGAGCCGTGGAGCAACAGTAGTTCTTCACCTA
CCACTCTGTCAATCCTGTTTGCTTGGAGGGAGGCTCTAGTGTTCTCTCTCTCTTCCAAGGGAACAGCCAA
GGCAGAGAGGCTAGCAGTAAACGGGGTTTGGATCTGGTGCCTCTCTGAGAGCGTTTTTTCTAACCTTAGC
TTGAATGGCTTGTGCTGTGTGTTTTGTTTGGTTTGCCTTTTTAAAAAAGACCCCTGCAAAGCTGGGCCGT
GGTGGCGCACACCTTTTACCCCAGCACTCTGGAGGCAGAGGCAGGAGGATCTCTTGAGTTAGAAGCCAGC
TTGGTCTACTGAGTTCCAGGACAGCCTGGGCTACAGAGAGGCATGGTGGCGCACGCCTTTAGTCCCAGCA
CTTGGGAGGCAGAGGCAGGCGGATTTCTGAGTTCGAGGCCAGCCTGGTCTACAGAGTGAGTTCCAGGACA
GCCAGGGCTACACAGAGAAACCTTGTCTCAAAAAACCAAAAAAACCCAACACAACACAAAAACAAGACCC
CCCTCACACCACACTCATTTTCTCCCCTGGGGATGGACCACTCTCATTAGCTTTGTCCCTCTAACGTGGG
CCCCCTCTCCTTCCCTAGGTGGTGAAGAGATCGTATGGCAAGGCCTTCAAACTGTCCATCTCTGCTCTCT
TTGTGACACCCAAGACAGCTGGGGCCCAGGTGGTGCTGACCGATCAGGAGCTGCAGTTGTGGCCCAGTGA
TCTGGACAAGCCATCTGCCTCCGAGGGCCTGCCCCCAGGGAGCCGAGCTCACGTCACCCTAGGCTGTGCG
GCCGACGTGCAGCCAGTGCAGACGGGTCTTGACCTCTTAGATATTTTACAACAGGTGAAGGGGGGCAGCC
AAGGTGAGGCGGTGGGTGAGCTCCCCCGGGGCAAGCTCTATTCCCTGGGCAAAGGGCGATGGATGCTGAG
CCTGACTAAGAAGATGGAGGTCAAGGCCATCTTCACGGGGTACTATGGGAAGGGCAAACCTGTGCCCATA
CATGGCAGCCGGAAGGGGGGTGCCATGCAGATCTGCACCATCATCTGAGGGTTCACACCGCTGTACCACT
GTACCACTGCTGTGGGTGGACACAGCAACATGCCTTCCACCTGATTTTTAAGATTTTTTTTTTTTTTACT
CAAAGCTAACGTACCTGTAACTTTTTAGGAGTCTATAAAATAACTACCAGTTCCTTGTGCCCACCCTTTC
TTTTCCTCTTAATACTCAGCTCTCAACACAGGCGGGTGGGCAGAGGAAGATACCATTCAGGAACCTGGAC
CAGAGGTGGTAATAAAAGGGCTGGGTCATTGGGCCTGCAGCAGCATCTGCCCAGATGCATCCAAGAACTT
GTTAACCCTCCCTTAGCCCCTGCCCATCCAAAGCTGGGCACACGCCTCAGAGCTGCCACATGCTAGGGAC
AAACGCTACTGCCACAGGAAAACAGTGGTAGCTTAGAGAGGGCTCAGATCCCTGGGTGCCAGTTTTTTAA
TCACTATAGCCCTTGGGAAGGCTTGGGCTATTTACTGGGCGGTAAAAGGGGTTTACCTCTCACCTTTGGT
CATTCCTAAGTTCCTATCTCCTTCACACTGTACCTAGGGTGATGGTTTCGGTCAGGCCATCTCCAGCAGC
ATGTCTCTGCTGTGTGGATGGAGGCCAAGGTCAGAAAGATGCCCCGAGAAGCTCCAGCTTTGCTTTCTGC
CCAGCCCATCTTGCCTGCATGGTGCCTCTGGGTCTTTAAAGGAGGGTGGTAGGTACCAAGAACCAGTGGA
GGGGACCCTGAGCTGGCAAGAGTAAGCCCCATCTCCTGCTCTGGCAAAGCCTGGCTCCTGGAGTACCTCA
CTCTGCCTTCTGCTCAGTAACAGGACCTTGCTAATTGGGTTGTCACCCGAGAATCTGGGATGTATGCTCC
TACCCTCACCCTGTGTTGCCCACCTCAGCCACCACTAAGACTGATACTGAAATAAATCATGTTAATCCCA
GCTGTGTGCTA"""
X = random_deletion(original_seq, len(original_seq) - 400)

def generate_variations(X, k):
    X1 = random_insertion(X, k)
    X1 = random_deletion(X1, k)
    X1 = random_mutation(X1, k)
    X1 = repeat_subsequence(X1, 2 * k, k)
    X1 = delete_subsequence(X1, 400 - 2 * k, k)

    X2 = random_insertion(X, k)
    X2 = random_deletion(X2, k)
    X2 = random_mutation(X2, k)
    X2 = repeat_subsequence(X2, 400 - 2 * k, k)
    X2 = delete_subsequence(X2, 2 * k, k)

    return X1, X2

for k in [10, 20, 30, 40, 50]:
    X1, X2 = generate_variations(X, k)

    start_time_nw = time.time()
    aligner = PairwiseAligner()
    aligner.mode = 'global'
    alignment = aligner.align(X1, X2)
    nw_score = alignment.score
    duration_nw = time.time() - start_time_nw

    start_time_blast = time.time()
    result_handle = NCBIWWW.qblast("blastn", "nt", X1)
    blast_record = NCBIXML.read(result_handle)
    blast_score = max(hsp.score for alignment in blast_record.alignments for hsp in alignment.hsps)
    duration_blast = time.time() - start_time_blast

    print(f"k = {k}")
    print(f"Needleman-Wunsch score: {nw_score}, Time taken: {duration_nw} seconds")
    print(f"BLAST score: {blast_score}, Time taken: {duration_blast} seconds")
    print("-" * 40)


k = 10
Needleman-Wunsch score: 353.0, Time taken: 0.0015423297882080078 seconds
BLAST score: 53.0, Time taken: 121.66256999969482 seconds
----------------------------------------
k = 20
Needleman-Wunsch score: 314.0, Time taken: 0.003565073013305664 seconds
BLAST score: 55.0, Time taken: 79.76442694664001 seconds
----------------------------------------
k = 30
Needleman-Wunsch score: 285.0, Time taken: 0.002398967742919922 seconds
BLAST score: 58.0, Time taken: 79.77599716186523 seconds
----------------------------------------
k = 40
Needleman-Wunsch score: 281.0, Time taken: 0.0018768310546875 seconds
BLAST score: 58.0, Time taken: 320.03875207901 seconds
----------------------------------------


/Users/vidyaanildhulappanavar/miniconda3/lib/python3.10/site-packages/Bio/Blast/NCBIWWW.py:281: BiopythonWarning: BLAST request 82V3JDDB016 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


k = 50
Needleman-Wunsch score: 280.0, Time taken: 0.004008769989013672 seconds
BLAST score: 53.0, Time taken: 1654.576457977295 seconds
----------------------------------------


In [8]:
import random
import time
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import PairwiseAligner
from Bio.Blast import NCBIWWW, NCBIXML

def random_deletion(seq, num_deletions):
    seq = list(seq)
    for _ in range(num_deletions):
        del seq[random.randint(0, len(seq) - 1)]
    return ''.join(seq)

def random_insertion(seq, num_insertions):
    seq = list(seq)
    for _ in range(num_insertions):
        seq.insert(random.randint(0, len(seq) - 1), random.choice("ACGT"))
    return ''.join(seq)

def random_mutation(seq, num_mutations):
    seq = list(seq)
    for _ in range(num_mutations):
        seq[random.randint(0, len(seq) - 1)] = random.choice("ACGT")
    return ''.join(seq)

def repeat_subsequence(seq, start, length):
    subseq = seq[start:start + length]
    return seq[:start + length] + subseq + seq[start + length:]

def delete_subsequence(seq, start, length):
    return seq[:start] + seq[start + length:]

#Homo sapiens actin beta (ACTB), mRNA

original_seq = """ACCGCCGAGACCGCGTCCGCCCCGCGAGCACAGAGCCTCGCCTTTGCCGATCCGCCGCCCGTCCACACCC
GCCGCCAGCTCACCATGGATGATGATATCGCCGCGCTCGTCGTCGACAACGGCTCCGGCATGTGCAAGGC
CGGCTTCGCGGGCGACGATGCCCCCCGGGCCGTCTTCCCCTCCATCGTGGGGCGCCCCAGGCACCAGGGC
GTGATGGTGGGCATGGGTCAGAAGGATTCCTATGTGGGCGACGAGGCCCAGAGCAAGAGAGGCATCCTCA
CCCTGAAGTACCCCATCGAGCACGGCATCGTCACCAACTGGGACGACATGGAGAAAATCTGGCACCACAC
CTTCTACAATGAGCTGCGTGTGGCTCCCGAGGAGCACCCCGTGCTGCTGACCGAGGCCCCCCTGAACCCC
AAGGCCAACCGCGAGAAGATGACCCAGATCATGTTTGAGACCTTCAACACCCCAGCCATGTACGTTGCTA
TCCAGGCTGTGCTATCCCTGTACGCCTCTGGCCGTACCACTGGCATCGTGATGGACTCCGGTGACGGGGT
CACCCACACTGTGCCCATCTACGAGGGGTATGCCCTCCCCCATGCCATCCTGCGTCTGGACCTGGCTGGC
CGGGACCTGACTGACTACCTCATGAAGATCCTCACCGAGCGCGGCTACAGCTTCACCACCACGGCCGAGC
GGGAAATCGTGCGTGACATTAAGGAGAAGCTGTGCTACGTCGCCCTGGACTTCGAGCAAGAGATGGCCAC
GGCTGCTTCCAGCTCCTCCCTGGAGAAGAGCTACGAGCTGCCTGACGGCCAGGTCATCACCATTGGCAAT
GAGCGGTTCCGCTGCCCTGAGGCACTCTTCCAGCCTTCCTTCCTGGGCATGGAGTCCTGTGGCATCCACG
AAACTACCTTCAACTCCATCATGAAGTGTGACGTGGACATCCGCAAAGACCTGTACGCCAACACAGTGCT
GTCTGGCGGCACCACCATGTACCCTGGCATTGCCGACAGGATGCAGAAGGAGATCACTGCCCTGGCACCC
AGCACAATGAAGATCAAGATCATTGCTCCTCCTGAGCGCAAGTACTCCGTGTGGATCGGCGGCTCCATCC
TGGCCTCGCTGTCCACCTTCCAGCAGATGTGGATCAGCAAGCAGGAGTATGACGAGTCCGGCCCCTCCAT
CGTCCACCGCAAATGCTTCTAGGCGGACTATGACTTAGTTGCGTTACACCCTTTCTTGACAAAACCTAAC
TTGCGCAGAAAACAAGATGAGATTGGCATGGCTTTATTTGTTTTTTTTGTTTTGTTTTGGTTTTTTTTTT
TTTTTTGGCTTGACTCAGGATTTAAAAACTGGAACGGTGAAGGTGACAGCAGTCGGTTGGAGCGAGCATC
CCCCAAAGTTCACAATGTGGCCGAGGACTTTGATTGCACATTGTTGTTTTTTTAATAGTCATTCCAAATA
TGAGATGCGTTGTTACAGGAAGTCCCTTGCCATCCTAAAAGCCACCCCACTTCTCTCTAAGGAGAATGGC
CCAGTCCTCTCCCAAGTCCACACAGGGGAGGTGATAGCATTGCTTTCGTGTAAATTATGTAATGCAAAAT
TTTTTTAATCTTCGCCTTAATACTTTTTTATTTTGTTTTATTTTGAATGATGAGCCTTCGTGCCCCCCCT
TCCCCCTTTTTTGTCCCCCAACTTGAGATGTATGAAGGCTTTTGGTCTCCCTGGGAGTGGGTGGAGGCAG
CCAGGGCTTACCTGTACACTGACTTGAGACCAGTTGAATAAAAGTGCACACCTTAAAAATGA"""
X = random_deletion(original_seq, len(original_seq) - 400)

def generate_variations(X, k):
    X1 = random_insertion(X, k)
    X1 = random_deletion(X1, k)
    X1 = random_mutation(X1, k)
    X1 = repeat_subsequence(X1, 2 * k, k)
    X1 = delete_subsequence(X1, 400 - 2 * k, k)

    X2 = random_insertion(X, k)
    X2 = random_deletion(X2, k)
    X2 = random_mutation(X2, k)
    X2 = repeat_subsequence(X2, 400 - 2 * k, k)
    X2 = delete_subsequence(X2, 2 * k, k)

    return X1, X2

for k in [10, 20, 30, 40, 50]:
    X1, X2 = generate_variations(X, k)

    start_time_nw = time.time()
    aligner = PairwiseAligner()
    aligner.mode = 'global'
    alignment = aligner.align(X1, X2)
    nw_score = alignment.score
    duration_nw = time.time() - start_time_nw

    start_time_blast = time.time()
    result_handle = NCBIWWW.qblast("blastn", "nt", X1)
    blast_record = NCBIXML.read(result_handle)
    blast_score = max(hsp.score for alignment in blast_record.alignments for hsp in alignment.hsps)
    duration_blast = time.time() - start_time_blast

    print(f"k = {k}")
    print(f"Needleman-Wunsch score: {nw_score}, Time taken: {duration_nw} seconds")
    print(f"BLAST score: {blast_score}, Time taken: {duration_blast} seconds")
    print("-" * 40)


k = 10
Needleman-Wunsch score: 350.0, Time taken: 0.0019500255584716797 seconds
BLAST score: 57.0, Time taken: 123.46107602119446 seconds
----------------------------------------


/Users/vidyaanildhulappanavar/miniconda3/lib/python3.10/site-packages/Bio/Blast/NCBIWWW.py:281: BiopythonWarning: BLAST request 82X1NEB4016 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


k = 20
Needleman-Wunsch score: 321.0, Time taken: 0.002343893051147461 seconds
BLAST score: 56.0, Time taken: 1821.627818107605 seconds
----------------------------------------
k = 30
Needleman-Wunsch score: 284.0, Time taken: 0.0020842552185058594 seconds
BLAST score: 58.0, Time taken: 140.04442596435547 seconds
----------------------------------------
k = 40
Needleman-Wunsch score: 276.0, Time taken: 0.0041692256927490234 seconds
BLAST score: 51.0, Time taken: 379.43712306022644 seconds
----------------------------------------


/Users/vidyaanildhulappanavar/miniconda3/lib/python3.10/site-packages/Bio/Blast/NCBIWWW.py:281: BiopythonWarning: BLAST request 82ZAU5P9013 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


k = 50
Needleman-Wunsch score: 270.0, Time taken: 0.005053043365478516 seconds
BLAST score: 55.0, Time taken: 789.1498930454254 seconds
----------------------------------------
